
Character-Level LSTM in PyTorch

In [0]:

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
!pip install PyPDF2

    100% |████████████████████████████████| 81kB 2.4MB/s 
  Running setup.py bdist_wheel for PyPDF2 ... - \ done
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
import PyPDF2

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open(fn, 'r') as file:
     content = file.read()
  

Saving The subtle act of not giving a fuck.txt to The subtle act of not giving a fuck.txt
User uploaded file "The subtle act of not giving a fuck.txt" with length 318254 bytes


In [5]:
content[:200]

'\x0c\x0c\x0c\x0cCONTENTS\n\nC HAPTER 1: Don’t Try\nThe Feedback Loop from Hell\nThe Subtle Art of Not Giving a Fuck\nSo Mark, What the Fuck Is the Point of This Book Anyway?\nC HAPTER 2: Happiness Is a Problem\nThe Misa'

Tokenization

In [0]:
chars = tuple(set(content))

In [7]:
chars

('g',
 '7',
 '1',
 'W',
 'e',
 'C',
 '’',
 '3',
 '\x0c',
 'q',
 '(',
 '4',
 ')',
 'V',
 'p',
 '—',
 'S',
 'o',
 'h',
 'b',
 'j',
 ',',
 'K',
 '|',
 '8',
 '[',
 'D',
 'c',
 'Z',
 'J',
 '#',
 'O',
 '5',
 '\n',
 'F',
 '“',
 '.',
 'I',
 'v',
 'H',
 '&',
 '6',
 'i',
 'Y',
 'u',
 ';',
 'X',
 'R',
 'L',
 ']',
 ' ',
 'N',
 '/',
 '‘',
 'y',
 's',
 'w',
 '-',
 '9',
 'M',
 '?',
 '2',
 'x',
 'B',
 'a',
 'r',
 'ç',
 'A',
 'n',
 'z',
 'd',
 't',
 'U',
 ':',
 'é',
 '0',
 '!',
 'P',
 'Q',
 'à',
 '”',
 '–',
 'G',
 'f',
 'l',
 'k',
 'E',
 '©',
 'm',
 'T',
 '→')

In [0]:
int2char = dict(enumerate(chars))

In [9]:
int2char

{0: 'g',
 1: '7',
 2: '1',
 3: 'W',
 4: 'e',
 5: 'C',
 6: '’',
 7: '3',
 8: '\x0c',
 9: 'q',
 10: '(',
 11: '4',
 12: ')',
 13: 'V',
 14: 'p',
 15: '—',
 16: 'S',
 17: 'o',
 18: 'h',
 19: 'b',
 20: 'j',
 21: ',',
 22: 'K',
 23: '|',
 24: '8',
 25: '[',
 26: 'D',
 27: 'c',
 28: 'Z',
 29: 'J',
 30: '#',
 31: 'O',
 32: '5',
 33: '\n',
 34: 'F',
 35: '“',
 36: '.',
 37: 'I',
 38: 'v',
 39: 'H',
 40: '&',
 41: '6',
 42: 'i',
 43: 'Y',
 44: 'u',
 45: ';',
 46: 'X',
 47: 'R',
 48: 'L',
 49: ']',
 50: ' ',
 51: 'N',
 52: '/',
 53: '‘',
 54: 'y',
 55: 's',
 56: 'w',
 57: '-',
 58: '9',
 59: 'M',
 60: '?',
 61: '2',
 62: 'x',
 63: 'B',
 64: 'a',
 65: 'r',
 66: 'ç',
 67: 'A',
 68: 'n',
 69: 'z',
 70: 'd',
 71: 't',
 72: 'U',
 73: ':',
 74: 'é',
 75: '0',
 76: '!',
 77: 'P',
 78: 'Q',
 79: 'à',
 80: '”',
 81: '–',
 82: 'G',
 83: 'f',
 84: 'l',
 85: 'k',
 86: 'E',
 87: '©',
 88: 'm',
 89: 'T',
 90: '→'}

In [0]:
char2int = {ch: ii for ii, ch in int2char.items()}

In [11]:
char2int

{'\n': 33,
 '\x0c': 8,
 ' ': 50,
 '!': 76,
 '#': 30,
 '&': 40,
 '(': 10,
 ')': 12,
 ',': 21,
 '-': 57,
 '.': 36,
 '/': 52,
 '0': 75,
 '1': 2,
 '2': 61,
 '3': 7,
 '4': 11,
 '5': 32,
 '6': 41,
 '7': 1,
 '8': 24,
 '9': 58,
 ':': 73,
 ';': 45,
 '?': 60,
 'A': 67,
 'B': 63,
 'C': 5,
 'D': 26,
 'E': 86,
 'F': 34,
 'G': 82,
 'H': 39,
 'I': 37,
 'J': 29,
 'K': 22,
 'L': 48,
 'M': 59,
 'N': 51,
 'O': 31,
 'P': 77,
 'Q': 78,
 'R': 47,
 'S': 16,
 'T': 89,
 'U': 72,
 'V': 13,
 'W': 3,
 'X': 46,
 'Y': 43,
 'Z': 28,
 '[': 25,
 ']': 49,
 'a': 64,
 'b': 19,
 'c': 27,
 'd': 70,
 'e': 4,
 'f': 83,
 'g': 0,
 'h': 18,
 'i': 42,
 'j': 20,
 'k': 85,
 'l': 84,
 'm': 88,
 'n': 68,
 'o': 17,
 'p': 14,
 'q': 9,
 'r': 65,
 's': 55,
 't': 71,
 'u': 44,
 'v': 38,
 'w': 56,
 'x': 62,
 'y': 54,
 'z': 69,
 '|': 23,
 '©': 87,
 'à': 79,
 'ç': 66,
 'é': 74,
 '–': 81,
 '—': 15,
 '‘': 53,
 '’': 6,
 '“': 35,
 '”': 80,
 '→': 90}

In [0]:
import numpy as np
encoded = np.array([char2int[ch] for ch in content])

In [13]:

encoded[:100]

array([ 8,  8,  8,  8,  5, 31, 51, 89, 86, 51, 89, 16, 33, 33,  5, 50, 39,
       67, 77, 89, 86, 47, 50,  2, 73, 50, 26, 17, 68,  6, 71, 50, 89, 65,
       54, 33, 89, 18,  4, 50, 34,  4,  4, 70, 19, 64, 27, 85, 50, 48, 17,
       17, 14, 50, 83, 65, 17, 88, 50, 39,  4, 84, 84, 33, 89, 18,  4, 50,
       16, 44, 19, 71, 84,  4, 50, 67, 65, 71, 50, 17, 83, 50, 51, 17, 71,
       50, 82, 42, 38, 42, 68,  0, 50, 64, 50, 34, 44, 27, 85, 33])

Pre-Processing the data 

In [0]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [15]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [16]:
test_seq

array([[3, 5, 1]])

Make training mini batches

In [0]:

def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [19]:

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 8  8  8  8  5 31 51 89 86 51]
 [55 50 71 18 64 71 50 71 18  4]
 [50 42 71 50 56 64 55 68  6 71]
 [70 65 44  0 55 36 50 39  4 50]
 [71 18 50 18 42 55 50 65 42  0]
 [50 83 64 42 84 44 65  4 55 50]
 [54 50 71 17 33  0  4 71 50 88]
 [50 71 18 42 55 50 42 70  4 64]]

y
 [[ 8  8  8  5 31 51 89 86 51 89]
 [50 71 18 64 71 50 71 18  4 54]
 [42 71 50 56 64 55 68  6 71 50]
 [65 44  0 55 36 50 39  4 50 18]
 [18 50 18 42 55 50 65 42  0 18]
 [83 64 42 84 44 65  4 55 50 64]
 [50 71 17 33  0  4 71 50 88 54]
 [71 18 42 55 50 42 70  4 64 50]]


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [22]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(91, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=91, bias=True)
)


In [23]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2599... Val Loss: 3.2181
Epoch: 1/20... Step: 20... Loss: 3.1642... Val Loss: 3.1326
Epoch: 2/20... Step: 30... Loss: 3.1487... Val Loss: 3.1125
Epoch: 2/20... Step: 40... Loss: 3.1238... Val Loss: 3.1051
Epoch: 3/20... Step: 50... Loss: 3.1208... Val Loss: 3.1017
Epoch: 3/20... Step: 60... Loss: 3.1144... Val Loss: 3.0998
Epoch: 4/20... Step: 70... Loss: 3.1329... Val Loss: 3.0970
Epoch: 4/20... Step: 80... Loss: 3.1026... Val Loss: 3.0915
Epoch: 5/20... Step: 90... Loss: 3.0977... Val Loss: 3.0801
Epoch: 5/20... Step: 100... Loss: 3.0809... Val Loss: 3.0539
Epoch: 5/20... Step: 110... Loss: 3.0329... Val Loss: 3.0004
Epoch: 6/20... Step: 120... Loss: 2.9607... Val Loss: 2.9225
Epoch: 6/20... Step: 130... Loss: 2.8439... Val Loss: 2.8034
Epoch: 7/20... Step: 140... Loss: 2.7386... Val Loss: 2.7012
Epoch: 7/20... Step: 150... Loss: 2.6573... Val Loss: 2.6136
Epoch: 8/20... Step: 160... Loss: 2.5976... Val Loss: 2.5686
Epoch: 8/20... Step: 170... Loss:

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
       
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):

    net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [28]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annat at is the mithing all the persant a sucking to malies the probeen a feor in a lofe to make and then and they teel tore
with thimes a feor hor the peollion and thing about abit of a constron a fecals than that world in the wellitionaly, the
row angicte as that
in than they,
I would soutter around wish of to sumfont of the wases and a fack to bithis fat to me worle in the wank to bule to bick our of to beeling thorged or the sarf able and all of the paliens of that yee some oun a that this who a fect to to that the somestally. If seless is ander are a farther one the are a becansions. Thin is
way the mest on were an and wenter all the pontite to al weal the reand the fourt to so they wist were theye’s and stant and whith the reand of he distan and
mesicess of something the simes in werred
the ruct and stelf in with in to shat a same the rall some to that she wist out ally the thourens to ane were aly and the rust feantiles
in the ragient is a chacting.
Whones a the pussing world.
W